# School Enrollment Rates For California

School Enrollment Rates


In [1]:
from ambry import get_library
import censuslib.dataframe

# The CensusDataFrame has mehods for doing sums and ratios with 90% margins and
# relative standard errors. 
df_class = censuslib.dataframe.CensusDataFrame

l = get_library()
b = l.bundle('census.gov-acs-enrollment-p5ye2014-hdp-0.0.1')

def ca_state_pred(r):
    """Select California counties using USPS Post codes"""
    from geoid.civick import GVid
    return (r.stusab == 'ca' and GVid.parse(r.gvid, exception=False).level == 'state')


# Population under 18
u18 = b.dep('under18').analysis.dataframe(ca_state_pred, df_class=df_class).set_index('gvid')

# The file should not have dupes, but it does ... 
states_geo = b.dep('geofile40').analysis.dataframe(df_class=df_class)
states_geo = states_geo[(states_geo.state == 6) & (states_geo.logrecno == 1 )][['name','logrecno','component']]\
.drop_duplicates().copy().set_index('logrecno')



In [2]:
enroll = b.dep('enrollment').analysis.dataframe(ca_state_pred, df_class=df_class).drop('id',1)
enroll = enroll.set_index('logrecno').join(states_geo, rsuffix='_geo')\
         .join(u18.set_index('logrecno'), rsuffix='_u18').copy()
enroll = enroll[enroll.component == '00'].drop_duplicates()

In [4]:
df = states_geo[['name']].copy()
df.columns = ['state'] + list(df.columns)[1:]
df['county'] = None
df['gvid'] = enroll['gvid']

# Re-name a few columns and add them to the new dataframe
df['enrolled_prek'],df['enrolled_prek_m90']  = enroll['b14001003'], enroll['b14001003_m90']
df['3to4'], df['3to4_m90'] = enroll['b09001004'], enroll['b09001004_m90']

df['not_enrolled_prek'] = df['3to4'] - df['enrolled_prek'
                                         ]
df['enrolled_rate'], df['enrolled_rate_m90']  = df.ratio('enrolled_prek', '3to4')
df['enrolled_rate_rse'] = df['enrolled_rate_m90'] / 1.645 / df['enrolled_rate'] * 100

df['not_enrolled_rate'] = 1.0 - df['enrolled_rate']

not_enrolled = df[['state','county','gvid','enrolled_prek','not_enrolled_prek','enrolled_rate', 
                   'not_enrolled_rate', 'enrolled_rate_m90','enrolled_rate_rse']] \
    .sort('not_enrolled_rate', ascending=False)
    
not_enrolled.to_csv('not_enrolled_prek.csv')
not_enrolled


,state,county,gvid,enrolled_prek,not_enrolled_prek,enrolled_rate,not_enrolled_rate,enrolled_rate_m90,enrolled_rate_rse
logrecno,,,,,,,,,
1,California,None,0E06,590364,464217,0.56,0.44,0.004442,0.482241
